### Label Encoder

In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [6]:
data =pd.read_csv("iris.csv",index_col=0)

In [ ]:
data.tail()

In [8]:
#Changes does not affect the "data" dataframe
data1=data.copy()

In [ ]:
labelencoder = LabelEncoder() # we want to label the Species column i.e.y variable
data1.iloc[:, -1] = labelencoder.fit_transform(data1.iloc[:,-1]) # -1: consider last column

In [ ]:
data1

### One Hot Encoder

#### Using sklearn

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
#data2=pd.read_csv("iris.csv",index_col=0)
data2=data.copy()

In [ ]:
data2

In [25]:
# creating instance of one-hot-encoder
OHE = OneHotEncoder()# Specifies the way unknown categories are handled during transform.

In [27]:
# convert it to array - to columns
enc_df = pd.DataFrame(OHE.fit_transform(data2[['Species']]).toarray())

In [28]:
enc_df

0    1    2
0    1.0  0.0  0.0
1    1.0  0.0  0.0
2    1.0  0.0  0.0
3    1.0  0.0  0.0
4    1.0  0.0  0.0
..   ...  ...  ...
145  0.0  0.0  1.0
146  0.0  0.0  1.0
147  0.0  0.0  1.0
148  0.0  0.0  1.0
149  0.0  0.0  1.0

[150 rows x 3 columns]

In [ ]:
# merge with main df
data_final = data2.iloc[:,0:4].join(enc_df)
data_final

#### Using Pandas

In [30]:
import pandas as pd

In [31]:
#data3 =pd.read_csv("iris.csv",index_col=0)
data3=data.copy()

In [32]:
data_encoded=pd.get_dummies(data3)

In [ ]:
data_encoded

### IsolationForest

In [34]:
# Isolation Forest example - useful for anomaly detection along with outliers
from sklearn.ensemble import IsolationForest
import pandas as pd

In [ ]:

df=pd.DataFrame({
      'Marks':[50,80,82,95,800],
      'Time':[1,2,3,1.5,60]
})
df

In [ ]:
# what is outlier here?
#iso=IsolationForest(random_state=10)# expecting 10 % outlier in dataset
iso=IsolationForest(random_state=10,contamination=0.1)
iso.fit(df)
out=iso.predict(df)

In [ ]:
out # -1: outlier - last value is an outlier

In [ ]:
df['Score']=out
df

In [39]:
data =pd.read_csv("iris.csv",index_col=0)
data_encoded=pd.get_dummies(data)

In [ ]:
data_encoded

In [ ]:
# training the model
clf = IsolationForest(random_state=10,contamination=.01)
# contamination: how much percentage of outliers you are expecting in dataset
# eg. in health care domain it will be very low (1% or less) 0.01 or 0.001
clf.fit(data_encoded)

In [42]:
# predictions
y_pred_outliers = clf.predict(data_encoded)

In [43]:
#-1 for outliers and 1 for inliers.
y_pred_outliers

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [ ]:
data_encoded

In [46]:
## Let us add a  new data point which is outlier
data_encoded.loc[151]=[20,40,30,50,1,0,0]

In [ ]:
data_encoded

In [ ]:
# training the model
clf = IsolationForest(random_state=10,contamination=.01)
clf.fit(data_encoded)
# predictions
y_pred_outliers = clf.predict(data_encoded)
y_pred_outliers # last has -1 so its an outlier

In [ ]:
# data_encoded['scores']=clf.decision_function(data_encoded)

In [49]:
data_encoded['anomaly']=clf.predict(data_encoded.iloc[:,0:7])
# we can pinpoint those outliers exactly by applying this filtering score

In [ ]:
data_encoded

In [ ]:
#Print the outlier data points
data_encoded[data_encoded['anomaly']==-1]# scores are given by decision tree

### PPS score

In [ ]:
#install the package
!pip install ppscore

In [53]:
import ppscore as pps

In [ ]:
# weighted F1 Score concept
#  https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1

In [ ]:
data.head()

In [ ]:
# Syntax: pps.score(df, "feature_column", "target_column")
pps.score(data, "Sepal.Length", "Petal.Length") # ppscore:0.55 so ok kind of score

In [56]:
#calculate the whole PPS matrix
pps.matrix(data)# petal width with species is most imp varaible with pps score 0.9276

x             y   ppscore            case  is_valid_score  \
0   Sepal.Length  Sepal.Length  1.000000  predict_itself            True   
1   Sepal.Length   Sepal.Width  0.000000      regression            True   
2   Sepal.Length  Petal.Length  0.550423      regression            True   
3   Sepal.Length   Petal.Width  0.431739      regression            True   
4   Sepal.Length       Species  0.471649  classification            True   
5    Sepal.Width  Sepal.Length  0.006966      regression            True   
6    Sepal.Width   Sepal.Width  1.000000  predict_itself            True   
7    Sepal.Width  Petal.Length  0.172375      regression            True   
8    Sepal.Width   Petal.Width  0.132858      regression            True   
9    Sepal.Width       Species  0.156915  classification            True   
10  Petal.Length  Sepal.Length  0.525617      regression            True   
11  Petal.Length   Sepal.Width  0.052136      regression            True   
12  Petal.Length  Petal.Length  1.000000  predict_itself            True   
13  Petal.Length   Petal.Width  0.744945      regression            True   
14  Petal.Length       Species  0.884812  classification            True   
15   Petal.Width  Sepal.Length  0.383911      regression            True   
16   Petal.Width   Sepal.Width  0.254616      regression            True   
17   Petal.Width  Petal.Length  0.798274      regression            True   
18   Petal.Width   Petal.Width  1.000000  predict_itself            True   
19   Petal.Width       Species  0.927652  classification            True   
20       Species  Sepal.Length  0.409634      regression            True   
21       Species   Sepal.Width  0.194307      regression            True   
22       Species  Petal.Length  0.785393      regression            True   
23       Species   Petal.Width  0.755749      regression            True   
24       Species       Species  1.000000  predict_itself            True   

                 metric  baseline_score  model_score                     model  
0                  None        0.000000     1.000000                      None  
1   mean absolute error        0.330667     0.364704   DecisionTreeRegressor()  
2   mean absolute error        1.488667     0.669271   DecisionTreeRegressor()  
3   mean absolute error        0.644667     0.366339   DecisionTreeRegressor()  
4           weighted F1        0.353333     0.658333  DecisionTreeClassifier()  
5   mean absolute error        0.684667     0.679897   DecisionTreeRegressor()  
6                  None        0.000000     1.000000                      None  
7   mean absolute error        1.488667     1.232058   DecisionTreeRegressor()  
8   mean absolute error        0.644667     0.559017   DecisionTreeRegressor()  
9           weighted F1        0.353333     0.454805  DecisionTreeClassifier()  
10  mean absolute error        0.684667     0.324794   DecisionTreeRegressor()  
11  mean absolute error        0.330667     0.313427   DecisionTreeRegressor()  
12                 None        0.000000     1.000000                      None  
13  mean absolute error        0.644667     0.164425   DecisionTreeRegressor()  
14          weighted F1        0.353333     0.925512  DecisionTreeClassifier()  
15  mean absolute error        0.684667     0.421816   DecisionTreeRegressor()  
16  mean absolute error        0.330667     0.246474   DecisionTreeRegressor()  
17  mean absolute error        1.488667     0.300302   DecisionTreeRegressor()  
18                 None        0.000000     1.000000                      None  
19          weighted F1        0.353333     0.953215  DecisionTreeClassifier()  
20  mean absolute error        0.684667     0.404204   DecisionTreeRegressor()  
21  mean absolute error        0.330667     0.266416   DecisionTreeRegressor()  
22  mean absolute error        1.488667     0.319479   DecisionTreeRegressor()  
23  mean absolute error        0.644667     0.157460   DecisionTreeRegressor()  
24           

In [57]:
df=pd.DataFrame(pps.matrix(data))
df[df.case!='predict_itself'].iloc[:,[0,1,2,3,5,8]].sort_values('ppscore',ascending=False)
# PPScore is highest in petal.width and species i.e. 0.927652 i.e. petal width is very good feature in predicting species

x             y   ppscore            case               metric  \
19   Petal.Width       Species  0.927652  classification          weighted F1   
14  Petal.Length       Species  0.884812  classification          weighted F1   
17   Petal.Width  Petal.Length  0.798274      regression  mean absolute error   
22       Species  Petal.Length  0.785393      regression  mean absolute error   
23       Species   Petal.Width  0.755749      regression  mean absolute error   
13  Petal.Length   Petal.Width  0.744945      regression  mean absolute error   
2   Sepal.Length  Petal.Length  0.550423      regression  mean absolute error   
10  Petal.Length  Sepal.Length  0.525617      regression  mean absolute error   
4   Sepal.Length       Species  0.471649  classification          weighted F1   
3   Sepal.Length   Petal.Width  0.431739      regression  mean absolute error   
20       Species  Sepal.Length  0.409634      regression  mean absolute error   
15   Petal.Width  Sepal.Length  0.383911      regression  mean absolute error   
16   Petal.Width   Sepal.Width  0.254616      regression  mean absolute error   
21       Species   Sepal.Width  0.194307      regression  mean absolute error   
7    Sepal.Width  Petal.Length  0.172375      regression  mean absolute error   
9    Sepal.Width       Species  0.156915  classification          weighted F1   
8    Sepal.Width   Petal.Width  0.132858      regression  mean absolute error   
11  Petal.Length   Sepal.Width  0.052136      regression  mean absolute error   
5    Sepal.Width  Sepal.Length  0.006966      regression  mean absolute error   
1   Sepal.Length   Sepal.Width  0.000000      regression  mean absolute error   

                       model  
19  DecisionTreeClassifier()  
14  DecisionTreeClassifier()  
17   DecisionTreeRegressor()  
22   DecisionTreeRegressor()  
23   DecisionTreeRegressor()  
13   DecisionTreeRegressor()  
2    DecisionTreeRegressor()  
10   DecisionTreeRegressor()  
4   DecisionTreeClassifier()  
3    DecisionTreeRegressor()  
20   DecisionTreeRegressor()  
15   DecisionTreeRegressor()  
16   DecisionTreeRegressor()  
21   DecisionTreeRegressor()  
7    DecisionTreeRegressor()  
9   DecisionTreeClassifier()  
8    DecisionTreeRegressor()  
11   DecisionTreeRegressor()  
5    DecisionTreeRegressor()  
1    DecisionTreeRegressor()